In [2]:
import numpy as np

h = 0.7
G = 4.302e-9 # Mpc (km/s)**2 M_sun**-1
H0 = 100 # h km Mpc**-1 s**-1
M_collapse = 8e12  # Msun h**-1
redshift = 4.434
omega_matter = 0.3
omega_lambda = 0.7

In [26]:
# H(z)
def hubble_parameter():
    return H0 * h * np.sqrt(omega_matter * (1.0 + redshift)**3  + omega_lambda)

# Calculate the concentration expected at a redshift
# From Roberston+06
def halo_concentration(mass):
    return 9.0 * (mass / (M_collapse * h))**(-0.13) / (1.0 + redshift)

# Mass must be in Msun
def mass_to_vel(mass):
    # (mass / h) to follow Springel+05 Section 2.4
    
    return (10.0 * G * hubble_parameter() * (mass / h))**(1.0 / 3.0)

def circular_velocity(mass):
    return (mass / 102.329)**0.26178 * (hubble_parameter() / H0)**(1/3)

def muratov_wind_vel(v_circ):
    return 0.854 * v_circ**1.12

def muratov_mass_loading(mass):
    return 3.55 * (mass / 1e10)**(-0.351)

def romeel_circ_vel(baryon_mass):
    return (baryon_mass / 102.329)**0.26178 * (hubble_parameter() / (H0 * h))**(1 / 3)

def romeel_wind_vel(circ_vel):
    return 2.0 * (circ_vel / 200)**0.12 * circ_vel

In [4]:
# In 1e10 Msun
gas_masses = np.asarray([12, 11.2, 6.7, 8.4, 4.8, 3.4, 1.6, 4.4, 2.2, 2.2, 3.1, 3.3, 1.2, 1.0])
gas_masses *= 1e10  # to actual Msun

In [5]:
gas_frac = 0.7

stellar_masses = (1.0 / gas_frac - 1.0) * gas_masses

print('Stellar masses in 1e10 units:')
print()

for stellar_mass in stellar_masses:
    print('%.3f' % float(stellar_mass / 1e10))

Stellar masses in 1e10 units:

5.143
4.800
2.871
3.600
2.057
1.457
0.686
1.886
0.943
0.943
1.329
1.414
0.514
0.429


In [6]:
# 1e10 Msun
halo_masses = stellar_masses * 100.0  # 100 factor from Behroozi et al. (2013) abundance matching for z ~ 4 - 5

print('Halo masses in 1e10 units:')
print()

for halo_mass in halo_masses:
    print('%.3f' % float(halo_mass / 1e10))

Halo masses in 1e10 units:

514.286
480.000
287.143
360.000
205.714
145.714
68.571
188.571
94.286
94.286
132.857
141.429
51.429
42.857


In [7]:
concentrations = halo_concentration(halo_masses)
concentrations

array([1.67467578, 1.68976363, 1.8064859 , 1.75415511, 1.88652663,
       1.97302252, 2.17614938, 1.90798718, 2.08789836, 2.08789836,
       1.99685845, 1.98069447, 2.25907547, 2.31325922])

In [8]:
velocities = mass_to_vel(halo_masses)
for velocity in velocities:
    print(velocity)

536.7227102815043
524.5202152954802
441.9581606747484
476.5582422850822
395.4606741425401
352.5189995498747
274.1971169790327
384.15555306378286
304.90446452643715
304.90446452643715
341.8299576355613
349.02848316150977
249.12461387890173
234.43523734112125


In [9]:
circ_vels = circular_velocity(gas_masses + stellar_masses)
for circ_vel in circ_vels:
    print(circ_vel)
    
print('Average circular velocity: %g km/s' % np.mean(circ_vels))

441.06764126069737
433.1730454901621
378.6578741644662
401.749145794726
347.0023864978458
317.04994445003115
260.2745232157458
339.18776027449036
282.9023401403068
282.9023401403068
309.4751590006811
314.5818844755372
241.39329180958984
230.1426611361303
Average circular velocity: 327.111 km/s


In [10]:
wind_vels = muratov_wind_vel(circ_vels)
for wind_vel in wind_vels:
    print(wind_vel)

782.1745688551798
766.511484390763
659.3173465953854
704.5104318063949
597.9023050589107
540.4068855234541
433.25275141620523
582.8420318604144
475.65360802177463
475.65360802177463
525.9673198594902
535.697507346891
398.2081015056708
377.4806175555523


In [11]:
mass_loading = muratov_mass_loading(np.sum(stellar_masses))
mass_loading

1.1012540241377462

In [12]:
print('Average wind velocity: %g km/s' % np.median(wind_vels))
print('Average mass loading: %g' % np.median(mass_loading))

Average wind velocity: 538.052 km/s
Average mass loading: 1.10125


In [13]:
# 3.24e-20 Mpc/km, 3e7 s/yr, 1e9 yr/Gyr
two_percent_hubble_time = 0.02 / (hubble_parameter() * 3.24e-20 * 3e7 * 1e9)
two_percent_hubble_time

0.042062069995242944

In [16]:
total_circ_vel = circular_velocity(np.sum(gas_masses) + np.sum(stellar_masses))
total_circ_vel

687.7851235899547

In [15]:
wind_vel = muratov_wind_vel(total_circ_vel)
wind_vel

1286.4857561141196

In [18]:
print('%g Msun' % float(np.sum(gas_masses) + np.sum(stellar_masses)))

9.35714e+11 Msun


In [28]:
circ_vel_romeel = romeel_circ_vel(np.sum(gas_masses) + np.sum(stellar_masses))
circ_vel_romeel

774.61653764383

In [29]:
wind_vel_romeel = romeel_wind_vel(circ_vel_romeel)
wind_vel_romeel

1822.5672942678343